In [1]:
# 读取pickle文件
import pickle
import numpy as np
import pandas as pd
from skimage.filters.rank import threshold

In [2]:
# 读取pickle文件
def read_pickle(file_path):
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
    return data

In [3]:
# Load Data
AIndexCSI500Weight = read_pickle('AIndexCSI500Weight.pkl')
# AIndexEODPrices_300_500_1000 = read_pickle('AIndexEODPrices_300_500_1000.pkl')
AIndexCSI1000Weight = read_pickle('AIndexCSI1000Weight.pkl')
AIndexHS300CloseWeight = read_pickle('AIndexHS300CloseWeight.pkl')
# AIndexMembers = read_pickle('AIndexMembers.pkl')
AShareEODPrices = read_pickle('AShareEODPrices.pkl')
# AShareTTMHis = read_pickle('AShareTTMHis.pkl')
# ChinaETFPchRedmList = read_pickle('ChinaETFPchRedmList.pkl')
AShareDescription = read_pickle('AShareDescription.pkl')
AShareEODDerivativeIndicator = read_pickle('AShareEODDerivativeIndicator.pkl')

In [4]:
AShareEODPrices['TRADE_DT'] = pd.to_datetime(AShareEODPrices['TRADE_DT'], format='%Y%m%d')
AIndexHS300CloseWeight['TRADE_DT'] = pd.to_datetime(AIndexHS300CloseWeight['TRADE_DT'], format='%Y%m%d')
AIndexCSI500Weight['TRADE_DT'] = pd.to_datetime(AIndexCSI500Weight['TRADE_DT'], format='%Y%m%d')
AIndexCSI1000Weight['TRADE_DT'] = pd.to_datetime(AIndexCSI1000Weight['TRADE_DT'], format='%Y%m%d')

In [5]:
# 生成沪深300样本空间函数
def GnerateSampleSpace(EODPrices, EODDerivativeIndicator, end_date, Index):
    end_date = end_date
    one_year_ago = (pd.to_datetime(end_date) - pd.DateOffset(years=1)).strftime('%Y%m%d')
    one_quarter_ago = (pd.to_datetime(end_date) - pd.DateOffset(months=3)).strftime('%Y%m%d')
    # 提取上一年的成分股名单
    if(Index == 'HS300'):
        Index_members = AIndexHS300CloseWeight[AIndexHS300CloseWeight['TRADE_DT'] >= one_year_ago]
    elif(Index == 'CSI500'):
        Index_members = AIndexCSI500Weight[AIndexCSI500Weight['TRADE_DT'] >= one_year_ago]
    elif(Index == 'CSI1000'):
        Index_members = AIndexCSI1000Weight[AIndexCSI1000Weight['TRADE_DT'] >= one_year_ago]
    else:
        print('Index Error!')
        return None
    if (Index == 'HS300' or Index == 'CSI500'):
        # 筛选出上市时间一年以上的科创板股票
        kc = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '科创板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_year_ago)
        ]
        # 筛选出上市时间一年以上的创业板股票
        cy = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '创业板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_year_ago)
        ]
        # 筛选出上市时间一个季度以上的主板股票
        zb = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '主板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_quarter_ago)
        ]
        # 筛选出上市时间一个季度以上的中小板股票
        zx = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '中小板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_quarter_ago)
        ]
        # 整合所有筛选结果
        merge_stocks = pd.concat([kc, cy, zb, zx])
    elif (Index == 'CSI1000'):
        # 筛选出上市时间一年以上的科创板股票
        kc = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '科创板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_year_ago)
        ]
        # 筛选出上市时间一个季度以上的创业板股票
        cy = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '创业板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_quarter_ago)
        ]
        # 筛选出上市时间一个季度以上的主板股票
        zb = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '主板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_quarter_ago)
        ]
        # 筛选出上市时间一个季度以上的中小板股票
        zx = AShareDescription[
            (AShareDescription['S_INFO_LISTBOARDNAME'] == '中小板') &
            (AShareDescription['S_INFO_LISTDATE'] < one_quarter_ago)
        ]
        # 整合所有筛选结果
        merge_stocks = pd.concat([kc, cy, zb, zx])
    # 样本空间
    EODPrices_merge = EODPrices[EODPrices['S_INFO_WINDCODE'].isin(merge_stocks['S_INFO_WINDCODE'])]
    EODDerivativeIndicator_merge = EODDerivativeIndicator[EODDerivativeIndicator['S_INFO_WINDCODE'].isin(merge_stocks['S_INFO_WINDCODE'])]
    # 近一年的数据
    EODPrices_merge = EODPrices_merge[EODPrices_merge['TRADE_DT'] >= one_year_ago]
    EODDerivativeIndicator_merge = EODDerivativeIndicator_merge[EODDerivativeIndicator_merge['TRADE_DT'] >= one_year_ago]
    # 样本空间
    # 计算每个股票的平均成交额
    dq_AMOUNT_AVG = EODPrices_merge.groupby('S_INFO_WINDCODE')['S_DQ_AMOUNT'].mean().reset_index()
    # 根据平均成交额进行排序
    dq_AMOUNT_AVG = dq_AMOUNT_AVG.sort_values(by='S_DQ_AMOUNT', ascending=False)
    if(Index == 'HS300'):
        threshold = 0.5
    elif(Index == 'CSI500'):
        threshold = 0.8
    elif(Index == 'CSI1000'):
        threshold = 0.8
    dq_AMOUNT_AVG = dq_AMOUNT_AVG.iloc[:int(len(dq_AMOUNT_AVG) * threshold)]  # 剔除成交额后50%/20%/20%的股票
    EODPrices_merge = EODPrices_merge[EODPrices_merge['S_INFO_WINDCODE'].isin(dq_AMOUNT_AVG['S_INFO_WINDCODE'])]
    # 计算新样本中日TOTAL_MV
    tot_MV_AVG = EODDerivativeIndicator_merge.groupby('S_INFO_WINDCODE')['S_VAL_MV'].mean().reset_index()
    # 筛选交易量前50%/80%/80%的股票
    tot_MV_AVG = tot_MV_AVG[tot_MV_AVG['S_INFO_WINDCODE'].isin(dq_AMOUNT_AVG['S_INFO_WINDCODE'])]
    # 匹配到股票的市值，将市值合并到AShareEODPrices_merge中
    EODPrices_merge = EODPrices_merge.merge(tot_MV_AVG, on='S_INFO_WINDCODE', how='left')
    EODPrices_merge = EODPrices_merge.sort_values(by=['S_VAL_MV'], ascending=False)
    # 返回样本空间
    return EODPrices_merge, EODDerivativeIndicator_merge, Index_members

In [6]:
# 预测成分股调整名单
def Predict(EODPrices_merge, EODDerivativeIndicator_merge, Index_members):
    # 选取样本空间中日均成交金额前60%的老样本股票进入下一步日均总市值的排名
    # 计算每个股票的平均成交额
    dq_AMOUNT_AVG = EODPrices_merge.groupby('S_INFO_WINDCODE')['S_DQ_AMOUNT'].mean().reset_index()
    dq_AMOUNT_AVG = dq_AMOUNT_AVG.sort_values(by='S_DQ_AMOUNT', ascending=False)

    # 根据指数类型设置参数
    if Index_members['S_INFO_WINDCODE'].iloc[1] == '000300.SH':  # 沪深300
        threshold = 0.6
        tot_1 = 240
        tot_2 = 360
        number = 30
        max_stocks = 300
    elif Index_members['S_INFO_WINDCODE'].iloc[1] == '000905.SH':  # 中证500
        threshold = 0.9
        tot_1 = 400
        tot_2 = 600
        number = 50
        max_stocks = 500
    elif Index_members['S_INFO_WINDCODE'].iloc[1] == '000852.SH':  # 中证1000
        threshold = 0.9
        tot_1 = 800
        tot_2 = 1200
        number = 100
        max_stocks = 1000

    # 剔除成交额低的股票
    dq_AMOUNT_AVG = dq_AMOUNT_AVG.iloc[:int(len(dq_AMOUNT_AVG) * threshold)]

    # 选取样本空间中日均成交金额后40%/10%/10%的老样本股票
    EODPrices_merge_old = EODPrices_merge[~EODPrices_merge['S_INFO_WINDCODE'].isin(dq_AMOUNT_AVG['S_INFO_WINDCODE'])]
    EODPrices_merge_old = EODPrices_merge_old[EODPrices_merge_old['S_INFO_WINDCODE'].isin(Index_members['S_CON_WINDCODE'])]
    EODPrices_merge_old_members = EODPrices_merge_old['S_INFO_WINDCODE'].unique()

    # 剔除样本空间中日均成交金额后40%/10%/10%的老样本股票
    EODPrices_merge = EODPrices_merge[~EODPrices_merge['S_INFO_WINDCODE'].isin(EODPrices_merge_old_members)]
    EODDerivativeIndicator_merge = EODDerivativeIndicator_merge[~EODDerivativeIndicator_merge['S_INFO_WINDCODE'].isin(EODPrices_merge_old_members)]

    # 市值排名在前240/400/800中的候选新样本优先进入指数
    tot_MV_AVG = EODDerivativeIndicator_merge.groupby('S_INFO_WINDCODE')['S_VAL_MV'].mean().reset_index()
    tot_MV_AVG = tot_MV_AVG.sort_values(by='S_VAL_MV', ascending=False)

    tot_small = tot_MV_AVG.iloc[:tot_1]
    tot_large = tot_MV_AVG.iloc[:tot_2]

    EODPrices_merge_small = EODPrices_merge[EODPrices_merge['S_INFO_WINDCODE'].isin(tot_small['S_INFO_WINDCODE'])]
    EODPrices_merge_large = EODPrices_merge[EODPrices_merge['S_INFO_WINDCODE'].isin(tot_large['S_INFO_WINDCODE'])]

    # 筛选排名前240/400/800的股票中不为成分股的股票
    new_candidates = EODPrices_merge[
        EODPrices_merge['S_INFO_WINDCODE'].isin(EODPrices_merge_small['S_INFO_WINDCODE']) &
        ~EODPrices_merge['S_INFO_WINDCODE'].isin(Index_members['S_CON_WINDCODE'])
    ]
    new_candidate_members = new_candidates['S_INFO_WINDCODE'].unique()[:number]

    # 市值排名在前360/600/1200中的候选老样本优先保留
    old_candidates = EODPrices_merge[
        EODPrices_merge['S_INFO_WINDCODE'].isin(EODPrices_merge_large['S_INFO_WINDCODE']) &
        EODPrices_merge['S_INFO_WINDCODE'].isin(Index_members['S_CON_WINDCODE'])
    ]
    old_candidate_members = old_candidates['S_INFO_WINDCODE'].unique()

    # 合并新老候选股票
    candidates_members = np.concatenate((old_candidate_members, new_candidate_members))

    # 若大于等于300/500/1000只股票，则只保留前300/500/1000只
    if len(candidates_members) >= max_stocks:
        candidates_members = candidates_members[:max_stocks]
    else:
        # 若不足300/500/1000只股票，则优先选择未进入候选股且总市值排名较前的股票
        not_in_candidates = tot_MV_AVG[~tot_MV_AVG['S_INFO_WINDCODE'].isin(candidates_members)]
        new_samples = not_in_candidates.sort_values(by='S_VAL_MV', ascending=False)['S_INFO_WINDCODE'].unique()
        diff = max_stocks - len(candidates_members)
        candidates_members = np.concatenate((candidates_members, new_samples[:diff]))

    return candidates_members

In [22]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
# 以半年为间隔预测成分股调整名单
def predict_half_yearly(start_date, end_date, Index):
    # 将日期字符串转换为 datetime 对象
    start_dt = datetime.strptime(start_date, '%Y%m%d')
    end_dt = datetime.strptime(end_date, '%Y%m%d')
    # 设计一个列表，用于保存所有预测结果
    all_results = []
    # 从起始日期开始，以半年为间隔，逐个预测成分股调整名单
    current_dt = start_dt
    while current_dt <= end_dt:
        # 定义本周期结束日期：当前日期加上6个月，然后减一天
        period_end_dt = current_dt + relativedelta(months=6) - timedelta(days=1)
        # 如果计算出的周期结束日期超过总体结束日期，则使用总体结束日期
        if period_end_dt > end_dt:
            period_end_dt = end_dt
        # 将当前周期的预测边界日期转换为字符串格式
        period_end_str = period_end_dt.strftime('%Y%m%d')
        # 设计一个字典，用于保存预测结果
        results = {
            "Predict_time": period_end_str,
            "Candidates_members_ID": []
        }
        # 生成当前周期的样本空间
        if(Index == 'HS300'):
            EODPrices_merge, EODDerivativeIndicator_merge, Index_members = GnerateSampleSpace(AShareEODPrices, AShareEODDerivativeIndicator, period_end_str, 'HS300')
        elif(Index == 'CSI500'):
            EODPrices_merge, EODDerivativeIndicator_merge, Index_members = GnerateSampleSpace(AShareEODPrices, AShareEODDerivativeIndicator, period_end_str, 'CSI500')
        elif(Index == 'CSI1000'):
            EODPrices_merge, EODDerivativeIndicator_merge, Index_members = GnerateSampleSpace(AShareEODPrices, AShareEODDerivativeIndicator, period_end_str, 'CSI1000')
        else:
            print('Index Error!')
            return None
        # 预测当前周期的成分股调整名单
        candidates_members = Predict(EODPrices_merge, EODDerivativeIndicator_merge, Index_members)
        # 将预测结果保存到结果字典中
        results["Candidates_members_ID"] = candidates_members
        # 将本周期预测结果保存到结果列表中
        all_results.append(results)
        # 更新下一周期的起始日期：本周期结束日期的下一天
        current_dt = period_end_dt + timedelta(days=1)
    return all_results

In [26]:
HS300_all_results = predict_half_yearly('20231101', '20241031', 'HS300')
CSI500_all_results = predict_half_yearly('20231101', '20241031', 'CSI500')
CSI1000_all_results = predict_half_yearly('20231101', '20241031', 'CSI1000')

In [27]:
HS300_all_results

[{'Predict_time': '20240430',
  'Candidates_members_ID': array(['600941.SH', '600519.SH', '601398.SH', '601939.SH', '601857.SH',
         '601288.SH', '601988.SH', '600938.SH', '601628.SH', '300750.SZ',
         '600036.SH', '601318.SH', '600028.SH', '601088.SH', '002594.SZ',
         '600900.SH', '000858.SZ', '601728.SH', '601658.SH', '601328.SH',
         '688981.SH', '000333.SZ', '601138.SH', '601899.SH', '601166.SH',
         '300760.SZ', '600030.SH', '601998.SH', '002415.SZ', '600276.SH',
         '601601.SH', '600809.SH', '601816.SH', '601319.SH', '600309.SH',
         '000568.SZ', '300059.SZ', '002475.SZ', '600690.SH', '600000.SH',
         '601668.SH', '603288.SH', '002714.SZ', '601633.SH', '000651.SZ',
         '601225.SH', '000001.SZ', '002352.SZ', '601766.SH', '688041.SH',
         '600406.SH', '601919.SH', '601818.SH', '003816.SZ', '601066.SH',
         '300999.SZ', '601888.SH', '603259.SH', '600887.SH', '601995.SH',
         '600104.SH', '600016.SH', '601985.SH', '601390.S

In [ ]:
# 生成HS300样本空间
EODPrices_merge_300, EODDerivativeIndicator_merge_300, Index_members_300 = GnerateSampleSpace(AShareEODPrices, AShareEODDerivativeIndicator, '20241031', 'HS300')

In [ ]:
# 生成CSI500样本空间
EODPrices_merge_500, EODDerivativeIndicator_merge_500, Index_members_500 = GnerateSampleSpace(AShareEODPrices, AShareEODDerivativeIndicator, '20241031', 'CSI500')

In [ ]:
# 生成CSI1000样本空间
EODPrices_merge_1000, EODDerivativeIndicator_merge_1000, Index_members_1000 = GnerateSampleSpace(AShareEODPrices, AShareEODDerivativeIndicator, '20241031', 'CSI1000')

In [ ]:
# 预测沪深300成分股调整名单
candidates_members_300 = Predict(EODPrices_merge_300, EODDerivativeIndicator_merge_300, Index_members_300)

In [ ]:
# 预测CSI500成分股调整名单
candidates_members_500 = Predict(EODPrices_merge_500, EODDerivativeIndicator_merge_500, Index_members_500)

In [ ]:
# 预测CSI1000成分股调整名单
candidates_members_1000 = Predict(EODPrices_merge_1000, EODDerivativeIndicator_merge_1000, Index_members_1000)